In [1]:
import sys
!{sys.executable} -m pip install awswrangler --index-url https://private.artifactory.foc.zone/api/pypi/rdf-pypi-virtual/simple --extra-index-url https://private.artifactory.foc.zone/artifactory/api/pypi/pypi-remote/simple

Looking in indexes: https://private.artifactory.foc.zone/api/pypi/rdf-pypi-virtual/simple, https://private.artifactory.foc.zone/artifactory/api/pypi/pypi-remote/simple
  Using cached https://private.artifactory.foc.zone/artifactory/api/pypi/pypi-remote/packages/packages/54/ec/3ea5f63f2811f74c40a22d8991218c4110c461afb61370b054e597fe55c8/awswrangler-2.16.1-py3-none-any.whl (248 kB)
  Using cached https://private.artifactory.foc.zone/artifactory/api/pypi/pypi-remote/packages/packages/3b/59/fbcbf9bdce46bc7a3ce6af140dadff9f9c8bd1a641e49ee0f1bfd21f830b/backoff-2.1.2-py3-none-any.whl (14 kB)
  Using cached https://private.artifactory.foc.zone/artifactory/api/pypi/pypi-remote/packages/packages/a2/b8/f202fe67a0f5a538a1d8a3cd5e7dc5f8aa8def03b44032fe1beabc45dc37/gremlinpython-3.6.0-py2.py3-none-any.whl (72 kB)
  Using cached https://private.artifactory.foc.zone/artifactory/api/pypi/pypi-remote/packages/packages/f6/17/5f50b5a47fe9988afcbc9cc63692ab7e34b3ddf8d608296212506c9d6f14/pyarrow-7.0.0-cp37-

In [2]:
import awswrangler as wr
import pandas as pd

# Some checks

In [3]:
sql_before = '''
SELECT DISTINCT 
       adp.mcvisid_visitorid,
       adp.post_tnt
FROM rktdp_adobe_omniture_raw_processed_access.adobe_dq_prcd_data AS adp
WHERE adp.post_tnt LIKE '%562585%' AND datekey<'2022-06-21' '''
df_before = wr.athena.read_sql_query(sql=sql_before, database="ktdp_model_outputs_raw_access", ctas_approach=False, 
                               workgroup='rcd-datascientist')

In [4]:
df_before.mcvisid_visitorid.nunique()

0

In [5]:
sql_after = '''
SELECT DISTINCT 
       adp.mcvisid_visitorid,
       adp.post_tnt
FROM rktdp_adobe_omniture_raw_processed_access.adobe_dq_prcd_data AS adp
WHERE adp.post_tnt LIKE '%562585%' AND datekey>='2022-06-21' '''
df_after = wr.athena.read_sql_query(sql=sql_after, database="ktdp_model_outputs_raw_access", ctas_approach=False, 
                                    workgroup='rcd-datascientist')

In [6]:
df_after.mcvisid_visitorid.nunique()

12394

In [7]:
sql_control = '''
SELECT DISTINCT 
       mcvisid_visitorid,
       post_tnt,
       datekey,
       hour
FROM rktdp_adobe_omniture_raw_processed_access.adobe_dq_prcd_data
WHERE (post_tnt='562585:0:0' OR post_tnt LIKE '%--562585:0:0%' OR post_tnt LIKE '562585:0:0%') AND datekey>='2022-06-21' '''
df_control = wr.athena.read_sql_query(sql=sql_control, database="ktdp_model_outputs_raw_access", ctas_approach=False, 
                                      workgroup='rcd-datascientist')

In [8]:
df_control.mcvisid_visitorid.nunique()

6382

In [9]:
sql_test = '''
SELECT DISTINCT 
       adp.mcvisid_visitorid,
       adp.post_tnt,
       adp.datekey,
       adp.hour
FROM rktdp_adobe_omniture_raw_processed_access.adobe_dq_prcd_data AS adp
WHERE (post_tnt='562585:1:0' OR post_tnt LIKE '%--562585:1:0%' OR post_tnt LIKE '562585:1:0%') AND datekey>='2022-06-21' '''
df_test = wr.athena.read_sql_query(sql=sql_test, database="ktdp_model_outputs_raw_access", ctas_approach=False, 
                                  workgroup='rcd-datascientist')

In [10]:
df_test.head()

,mcvisid_visitorid,post_tnt,datekey,hour
0,81528838810448043783116974624610335233,562585:1:0--**--562432:1:0--**--560145:0:0--**...,2022-06-26,4
1,82668918350976323174607153608867312638,562585:1:0--**--562432:0:0,2022-06-28,21
2,54439236892634191521763521880140678573,562585:1:0--**--562432:1:0,2022-06-28,22
3,04320741464401047131733508659505845333,562585:1:0,2022-06-30,19
4,35747565093530691483937630203632047020,562585:1:0,2022-07-05,20


In [11]:
df_test.mcvisid_visitorid.nunique()

6012

In [12]:
5295+4998

10293

# Checking events data

In [13]:
sql_event = '''
SELECT * 
FROM rktdp_adobe_omniture_raw_processed_access.adobe_websessionpostevent
WHERE mcvisid_visitorid = '49799992538523264713463772097096218572'
AND datekey >= '2022-07-01' AND eventid='247'
'''
df_event = wr.athena.read_sql_query(sql=sql_event, database="ktdp_model_outputs_raw_access", ctas_approach=False, 
                                    workgroup='rcd-datascientist')

In [14]:
df_event

,uniquevisitkey,calcualted_visitorid,mcvisid_visitorid,date_time,post_event_list,eventid,event_lookup,hash_value,datekey,hour


# Comparing event data

In [15]:
sql = '''
WITH all AS
(
    SELECT DISTINCT 
           adp.mcvisid_visitorid,
           adp.datekey,
           CASE
               WHEN post_tnt LIKE '%562585:0:0%' THEN '562585:0:0'
               WHEN post_tnt LIKE '%562585:1:0%' THEN '562585:1:0'
           END AS experiment_group
    FROM rktdp_adobe_omniture_raw_processed_access.adobe_dq_prcd_data AS adp
    WHERE adp.post_tnt LIKE '%562585%' AND adp.datekey>='2022-06-21' AND adp.datekey<='2022-07-12'
),

first AS
(
    SELECT mcvisid_visitorid, 
           experiment_group,
           MIN(datekey) AS datekey
    FROM all
    GROUP BY mcvisid_visitorid, experiment_group
),

event AS
(
    SELECT first.mcvisid_visitorid,
           first.experiment_group,
           first.datekey,
           ev.datekey AS ev_datekey
    FROM first
    LEFT JOIN rktdp_adobe_omniture_raw_processed_access.adobe_websessionpostevent AS ev
    ON first.mcvisid_visitorid = ev.mcvisid_visitorid
    AND ev.datekey>='2022-06-21' AND ev.datekey<='2022-07-12' AND ev.eventid = '228'
)

SELECT *
FROM event
'''
df = wr.athena.read_sql_query(sql=sql, database="rktdp_adobe_omniture_raw_processed_access", ctas_approach=False, 
                              workgroup='rcd-datascientist')

In [16]:
df.mcvisid_visitorid.nunique()

10470

In [17]:
df[['mcvisid_visitorid', 'experiment_group']].drop_duplicates().experiment_group.value_counts()

562585:0:0    5393
562585:1:0    5077
Name: experiment_group, dtype: Int64

In [18]:
df[~df.ev_datekey.isna()].experiment_group.value_counts()

562585:1:0    60
562585:0:0    11
Name: experiment_group, dtype: Int64

In [19]:
df[~df.ev_datekey.isna()][['mcvisid_visitorid', 'experiment_group']].drop_duplicates().experiment_group.value_counts()

562585:1:0    26
562585:0:0     4
Name: experiment_group, dtype: Int64

In [20]:
sql = '''
WITH all AS
(
    SELECT DISTINCT 
           adp.mcvisid_visitorid,
           adp.datekey,
           CASE
               WHEN post_tnt LIKE '%562585:0:0%' THEN '562585:0:0'
               WHEN post_tnt LIKE '%562585:1:0%' THEN '562585:1:0'
           END AS experiment_group
    FROM rktdp_adobe_omniture_raw_processed_access.adobe_dq_prcd_data AS adp
    WHERE adp.post_tnt LIKE '%562585%' AND adp.datekey>='2022-06-21' AND adp.datekey<='2022-07-12'
),

first AS
(
    SELECT mcvisid_visitorid, 
           experiment_group,
           MIN(datekey) AS datekey
    FROM all
    GROUP BY mcvisid_visitorid, experiment_group
),

event AS
(
    SELECT first.mcvisid_visitorid,
           first.experiment_group,
           first.datekey,
           ev.datekey AS ev_datekey,
           ev.post_event_list
    FROM first
    LEFT JOIN rktdp_adobe_omniture_raw_processed_access.adobe_dq_prcd_data AS ev
    ON first.mcvisid_visitorid = ev.mcvisid_visitorid
    AND ev.datekey>='2022-06-21' AND ev.datekey<='2022-07-12' AND (ev.post_event_list LIKE '%,228,%' OR ev.post_event_list LIKE '228,%'
    OR ev.post_event_list LIKE '%,228' OR ev.post_event_list LIKE '228')
)

SELECT *
FROM event
'''
df = wr.athena.read_sql_query(sql=sql, database="rktdp_adobe_omniture_raw_processed_access", ctas_approach=False, 
                              workgroup='rcd-datascientist')

In [21]:
df[['mcvisid_visitorid', 'experiment_group']].drop_duplicates().experiment_group.value_counts()

562585:0:0    5393
562585:1:0    5077
Name: experiment_group, dtype: Int64

In [22]:
df[~df.ev_datekey.isna()][['mcvisid_visitorid', 'experiment_group']].drop_duplicates().experiment_group.value_counts()

562585:1:0    26
562585:0:0     4
Name: experiment_group, dtype: Int64

# Functions

In [23]:
def df_sql_event_table(eventid):
    sql = f'''
    WITH all AS
    (
        SELECT DISTINCT 
               adp.mcvisid_visitorid,
               adp.datekey,
               CASE
                   WHEN post_tnt LIKE '%562585:0:0%' THEN '562585:0:0'
                   WHEN post_tnt LIKE '%562585:1:0%' THEN '562585:1:0'
               END AS experiment_group
        FROM rktdp_adobe_omniture_raw_processed_access.adobe_dq_prcd_data AS adp
        WHERE adp.post_tnt LIKE '%562585%' AND adp.datekey>='2022-06-21' AND adp.datekey<='2022-07-12'
    ),

    first AS
    (
        SELECT mcvisid_visitorid, 
               experiment_group,
               MIN(datekey) AS datekey
        FROM all
        GROUP BY mcvisid_visitorid, experiment_group
    ),

    event AS
    (
        SELECT first.mcvisid_visitorid,
               first.experiment_group,
               first.datekey,
               ev.datekey AS ev_datekey
        FROM first
        LEFT JOIN rktdp_adobe_omniture_raw_processed_access.adobe_websessionpostevent AS ev
        ON first.mcvisid_visitorid = ev.mcvisid_visitorid
        AND ev.datekey>='2022-06-21' AND ev.datekey<='2022-07-12' AND ev.eventid = '{eventid}'
    )

    SELECT *
    FROM event
    '''
    return sql

In [24]:
def df_sql_event_list(eventid):
    sql = f'''
    WITH all AS
    (
        SELECT DISTINCT 
               adp.mcvisid_visitorid,
               adp.datekey,
               CASE
                   WHEN post_tnt LIKE '%562585:0:0%' THEN '562585:0:0'
                   WHEN post_tnt LIKE '%562585:1:0%' THEN '562585:1:0'
               END AS experiment_group
        FROM rktdp_adobe_omniture_raw_processed_access.adobe_dq_prcd_data AS adp
        WHERE adp.post_tnt LIKE '%562585%' AND adp.datekey>='2022-06-21' AND adp.datekey<='2022-07-12'
    ),

    first AS
    (
        SELECT mcvisid_visitorid, 
               experiment_group,
               MIN(datekey) AS datekey
        FROM all
        GROUP BY mcvisid_visitorid, experiment_group
    ),

    event AS
    (
        SELECT first.mcvisid_visitorid,
               first.experiment_group,
               first.datekey,
               ev.datekey AS ev_datekey,
               ev.post_event_list
        FROM first
        LEFT JOIN rktdp_adobe_omniture_raw_processed_access.adobe_dq_prcd_data AS ev
        ON first.mcvisid_visitorid = ev.mcvisid_visitorid
        AND ev.datekey>='2022-06-21' AND ev.datekey<='2022-07-12' AND (ev.post_event_list LIKE '%,{eventid},%' 
        OR ev.post_event_list LIKE '{eventid},%'
        OR ev.post_event_list LIKE '%,{eventid}' OR ev.post_event_list LIKE '{eventid}')
    )

    SELECT *
    FROM event
    '''
    return sql

# Events

In [25]:
HARD_ONLINE_CREDIT_PULL = 228
LEAD_FORM_SUBMISSION = 206
APPLICATION_START = 250

# Hard online credit pull

## EVent table approach

In [26]:
df = wr.athena.read_sql_query(sql=df_sql_event_table(HARD_ONLINE_CREDIT_PULL), 
                              database="rktdp_adobe_omniture_raw_processed_access", ctas_approach=False, 
                              workgroup='rcd-datascientist')

In [27]:
# Total test population
df[['mcvisid_visitorid', 'experiment_group']].drop_duplicates().experiment_group.value_counts()

562585:0:0    5393
562585:1:0    5077
Name: experiment_group, dtype: Int64

In [28]:
# duplicate credit pulls
df[~df.ev_datekey.isna()].experiment_group.value_counts()

562585:1:0    60
562585:0:0    11
Name: experiment_group, dtype: Int64

In [29]:
# Deduped pulls
df[~df.ev_datekey.isna()][['mcvisid_visitorid', 'experiment_group']].drop_duplicates().experiment_group.value_counts()

562585:1:0    26
562585:0:0     4
Name: experiment_group, dtype: Int64

## Event list approach

In [30]:
df = wr.athena.read_sql_query(sql=df_sql_event_list(HARD_ONLINE_CREDIT_PULL), 
                              database="rktdp_adobe_omniture_raw_processed_access", ctas_approach=False, 
                              workgroup='rcd-datascientist')

In [31]:
# Total test population
df[['mcvisid_visitorid', 'experiment_group']].drop_duplicates().experiment_group.value_counts()

562585:0:0    5393
562585:1:0    5077
Name: experiment_group, dtype: Int64

In [32]:
# duplicate credit pulls
df[~df.ev_datekey.isna()].experiment_group.value_counts()

562585:1:0    96
562585:0:0    11
Name: experiment_group, dtype: Int64

In [33]:
# Deduped pulls
df[~df.ev_datekey.isna()][['mcvisid_visitorid', 'experiment_group']].drop_duplicates().experiment_group.value_counts()

562585:1:0    26
562585:0:0     4
Name: experiment_group, dtype: Int64

# Lead Form Submission

## Event table approach

In [34]:
df = wr.athena.read_sql_query(sql=df_sql_event_table(LEAD_FORM_SUBMISSION), 
                              database="rktdp_adobe_omniture_raw_processed_access", ctas_approach=False, 
                              workgroup='rcd-datascientist')

In [35]:
# Total test population
df[['mcvisid_visitorid', 'experiment_group']].drop_duplicates().experiment_group.value_counts()

562585:0:0    5393
562585:1:0    5077
Name: experiment_group, dtype: Int64

In [36]:
# duplicate leads
df[~df.ev_datekey.isna()].experiment_group.value_counts()

562585:0:0    3291
562585:1:0     216
Name: experiment_group, dtype: Int64

In [37]:
# Deduped leads
df[~df.ev_datekey.isna()][['mcvisid_visitorid', 'experiment_group']].drop_duplicates().experiment_group.value_counts()

562585:0:0    3041
562585:1:0     192
Name: experiment_group, dtype: Int64

## Event list approach

In [38]:
df = wr.athena.read_sql_query(sql=df_sql_event_list(LEAD_FORM_SUBMISSION), 
                              database="rktdp_adobe_omniture_raw_processed_access", ctas_approach=False, 
                              workgroup='rcd-datascientist')

In [39]:
# Total test population
df[['mcvisid_visitorid', 'experiment_group']].drop_duplicates().experiment_group.value_counts()

562585:0:0    5393
562585:1:0    5077
Name: experiment_group, dtype: Int64

In [40]:
# duplicate leads
df[~df.ev_datekey.isna()].experiment_group.value_counts()

562585:0:0    3291
562585:1:0     216
Name: experiment_group, dtype: Int64

In [41]:
# Deduped leads
df[~df.ev_datekey.isna()][['mcvisid_visitorid', 'experiment_group']].drop_duplicates().experiment_group.value_counts()

562585:0:0    3041
562585:1:0     192
Name: experiment_group, dtype: Int64

# Application Start

## Event table approach

In [42]:
df = wr.athena.read_sql_query(sql=df_sql_event_table(APPLICATION_START), 
                              database="rktdp_adobe_omniture_raw_processed_access", ctas_approach=False, 
                              workgroup='rcd-datascientist')

In [43]:
# Total test population
df[['mcvisid_visitorid', 'experiment_group']].drop_duplicates().experiment_group.value_counts()

562585:0:0    5393
562585:1:0    5077
Name: experiment_group, dtype: Int64

In [44]:
# duplicate application
df[~df.ev_datekey.isna()].experiment_group.value_counts()

562585:1:0    1818
562585:0:0      92
Name: experiment_group, dtype: Int64

In [45]:
# Deduped application
df[~df.ev_datekey.isna()][['mcvisid_visitorid', 'experiment_group']].drop_duplicates().experiment_group.value_counts()

562585:1:0    1713
562585:0:0      84
Name: experiment_group, dtype: Int64

# Event list approach

In [46]:
df = wr.athena.read_sql_query(sql=df_sql_event_list(APPLICATION_START), 
                              database="rktdp_adobe_omniture_raw_processed_access", ctas_approach=False, 
                              workgroup='rcd-datascientist')

In [47]:
# Total test population
df[['mcvisid_visitorid', 'experiment_group']].drop_duplicates().experiment_group.value_counts()

562585:0:0    5393
562585:1:0    5077
Name: experiment_group, dtype: Int64

In [48]:
# duplicate application
df[~df.ev_datekey.isna()].experiment_group.value_counts()

562585:1:0    1818
562585:0:0      92
Name: experiment_group, dtype: Int64

In [49]:
# Deduped application
df[~df.ev_datekey.isna()][['mcvisid_visitorid', 'experiment_group']].drop_duplicates().experiment_group.value_counts()

562585:1:0    1713
562585:0:0      84
Name: experiment_group, dtype: Int64